In [3]:
pip install pandas psycopg2

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------- ----------------------------- 2.9/11.0 MB 21.1 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.0 MB 11.5 MB/s eta 0:00:01
   -------------------- ------------------- 5.5/11.0 MB 10.0 MB/s eta 0:00:01
   ------------------------ --------------- 6.8/11.0 MB 8.4 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 7.8 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 7.1 MB/s eta 0:00:01
   -------------------------------- ------- 8.9/11.0 MB 6.5 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 6.0 MB/s eta 0:00:01
   ----------------------------------- ---- 9.7/11.0 MB 5.3 MB/s eta 0:00:01
   ------------------------------------- -- 10.2/11.0 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 4.6 MB/s  0:00:02
   


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import pandas as pd
import psycopg2
import os

conn = psycopg2.connect(
    dbname="f1",
    user="postgres",
    password="Theshield193",
    host="localhost"
)

sql = """
WITH weather_avg as (
    SELECT session_key, AVG(track_temperature) as avg_track_temperature
    FROM weather
    GROUP BY session_key
)
SELECT 
    l.lap_number,
    l.duration_sector_1,
    l.duration_sector_2,
    l.duration_sector_3,
    s.session_type,
    st.compound,
    st.tyre_age_at_start,
    w.avg_track_temperature as track_temperature,
    d.driver_number,
    l.session_key
FROM laps l
JOIN sessions s ON l.session_key = s.session_key
JOIN stint st ON l.session_key = st.session_key AND l.driver_number = st.driver_number and l.lap_number between st.lap_start and st.lap_end
JOIN weather_avg w ON l.session_key = w.session_key
JOIN driver d on l.driver_number = d.driver_number
WHERE s.session_type = 'Race'
ORDER BY l.lap_number, l.session_key
;
"""

# Load the sql data in a pandas dataframe
df = pd.read_sql(sql,conn)

df['compound'] = df['compound'].astype(str);
df['session_type'] = df['session_type'].astype(str);

df.head()

# df.groupby(['session_key', 'lap_number', 'driver_number']).size()

output_folder = "../data"

csv_path = os.path.join(output_folder, "f1_lap_training_csv")
print(f"Path: {csv_path}")
df.to_csv(csv_path, index=False)

print(f"✅ Dataset created: {csv_path}")

C:\Users\USER\AppData\Local\Temp\ipykernel_21152\3501701619.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,conn)


Path: ../data\f1_lap_training_csv
✅ Dataset created: ../data\f1_lap_training_csv
